## Mining the data

In [12]:
import pandas as pd
import datetime
from datetime import datetime,timedelta
import pandas_datareader.data as web
from IPython.display import clear_output

#### Pulling files for 11 years of daily news with timestamps fron reuters. The data was found on harvard university website as a study data set and stored in my public s3 bucket.

In [25]:
#downloading raw reuters news files from my public s3 bucket
url='https://capstone-project-bucket-niko.s3.amazonaws.com/reuters-newswire-2007.v5.csv'
df=pd.read_csv(url)
years = range(2008,2018)
for y in years:
    url='https://capstone-project-bucket-niko.s3.amazonaws.com/reuters-newswire-{}.v5.csv'.format(y)
    data=pd.read_csv(url)
    df=pd.concat([df,data])
    clear_output()
    print('Concatenated '+str(y+1)+' loop')

Concatenated 2017 loop


In [26]:
print('The data has '+str(len(df))+' rows.')
df.head(10)

The data has 16121309 rows.


,publish_time,headline_text
0,200701010000,George Michael rocks Wembley
1,200701010001,Placido Domingo backs NY Met's opera at cinema...
2,200701010004,Shopping gets more expensive
3,200701010012,Apple posts options expenses; stands by CEO Jobs
4,200701010012,Dolly for dinner? Not just yet; critics say
5,200701010012,Goodyear workers ratify three-year contract
6,200701010012,Stock rally of 2006 smiled on all sectors
7,200701010013,Fancy getting dirty?
8,200701010017,Washington pays tribute to Ford
9,200701010018,Forest Whitaker speaks out


#### We have 16M rows and hundreds of news for each day. We need to format time from unix to datetime, consider all news published prior to 9.30am (market open) as previous day news for modelling purposes  and concatenate all news for the same day in one row.

In [27]:
#Converting Unix Time to Datetime and setting s index
data=df.copy()
data['publish_time']=data['publish_time'].astype(str)
data['publish_time']= data['publish_time'].apply(lambda x:datetime.strptime(x, '%Y%m%d%H%M'))
data.columns=(['time','news'])
data=data.set_index('time')

In [28]:
#Ading news prior to 9.30am to previous day for modelling purposes
idx=[]
for i in data.index:
    if i.hour<9:
        idx.append(i - timedelta(days=1))
        
    elif i.hour==9 and i.minute<25:
        idx.append(i - timedelta(days=1))
    else:
        idx.append(i)
        
idx = [i.strftime('%Y-%m-%d') for i in idx]#converting to same format that will be used later when pulling stock data
data.index=idx
days=list(pd.DataFrame(idx,columns=['idx']).idx.unique())#making list of unique dates


In [31]:
#concatenating all news for same day into one row
news=[]
for day in days:
    daily_news=''
    frame=data[data.index==day]
    for headline in frame.news:
        daily_news+=str(headline)+' '
    news.append(daily_news)
    clear_output()
    print('Date '+day+' updated.')

Date 2017-12-31 updated.


In [32]:
#combining into  data frame
df=pd.DataFrame(columns=['Date','Headlines'], )
df['Date']=days
df['Headlines']=news
df.head()

,Date,Headlines
0,2006-12-31,George Michael rocks Wembley Placido Domingo b...
1,2007-01-01,Millions of Hindus due for holy bath in north ...
2,2007-01-02,Iraqi PM Maliki strengthened by Saddam executi...
3,2007-01-03,UPDATE 2-Ireland's CRH sees 2006 pretax profit...
4,2007-01-04,STOCKS NEWS EUROPE-Carnival steams ahead on br...


In [33]:
#inverting dataset for easir elimnination of weekends
df=df.sort_index(ascending=0)
df=df.reset_index(drop=True)
#creating a list of dates when the stock market was open
marketDates=list(web.DataReader('SPY','yahoo','2006-12-30','2017-12-31').reset_index()['Date'].apply(lambda x:datetime.strftime(x, '%Y-%m-%d')))
#moving data from closed days to previous day market was opwn
for i in range(len(df)-1):
    if df.Date[i] not in marketDates:
        df.Headlines[i+1]=df.Headlines[i]+df.Headlines[i+1]

In [34]:
#dropping the rows for dates the market was closed
df = pd.merge(df,pd.DataFrame(marketDates,columns=['Date']), on=['Date'])
df=df.sort_index(ascending=0)
df=df.reset_index(drop=True)

In [35]:
#setting index to date
df=df.set_index('Date')

In [37]:
#saving file
df.to_csv('newsData.csv', index=True)